# Python实现进化算法求解TSP（旅行商问题）

作者：Evan-He
链接：<https://zhuanlan.zhihu.com/p/153098599>

Ps: 本文为作者在学习书籍《Matlab 智能算法：30个案例分析 》时，根据自己理解，利用python复现进化算法（包括遗传算法，混合粒子群算法，模拟退火算法，蚁群算法），初学，大佬勿喷。

TSP(Traveling Salesman Problem,旅行商问题)：一个商品推销员要去若干个城市推销商品，该推销员从一个城市出发，需要经过所有城市后，回到出发地。应如何选择行进路线，以使总的行程最短。

对于n个城市的TSP，本文利用python分别实现遗传算法，混合粒子群算法，模拟退火算法，蚁群算法对该问题的求解。

## 一、遗传算法

### A. 遗传算法的基本运算过程如下：

1. 初始化编码：设置最大进化代数$T_{max}$、选择概率、交叉概率、变异概率、随机生成$m$个染色体的群体，每个染色体的编码对于一个可行的路径（如6个城市，$[1,3,2,6,4,5]$就是一条可行路径）。
1. 适应度函数：对每一个染色体$x_k$，其个体适应度函数设置为$f(x_k)=\frac{1}{D_k}$，其中$D_k$表示该条路径的总长度。
1. 选择：将旧群体中的染色体以一定概率选择到新群体，每条染色体选中的概率与对应的适应度函数只相对应，本文采用随机遍历选择。
1. 交叉：在交叉概率的控制下，对选择群体中的个体进行两两交叉。
1. 变异：在变异概率的控制下，对单个染色体随机交换两个点的位置。
1. 进化逆转：将选择的染色体随机选择两个位置$r_1:r_2$，将$r_1:r_2$的元素翻转为$r_2:r_1$，如果翻转后的适应度更高，则替换原染色体，否则不变。
1. 重插：选择的子代与父代结合，形成新的种群，循环操作。

### B. python程序实现

构建一个类保存遗传算法的初始化参数和函数计算


In [ ]:
from math import floor
from typing import TypeAlias, Any
import numpy as np
from numpy import float64, int64
import numpy.typing as npt
import matplotlib.pyplot as plt  # 导入所需要的库

In [ ]:
ArrayF: TypeAlias = npt.NDArray[float64]
ArrayI: TypeAlias = npt.NDArray[int64]

plt.style.use("seaborn-v0_8")

In [ ]:
class Gena_TSP:
    def __init__(
        self,
        data: ArrayF,
        maxgen: int = 200,
        size_pop: int = 200,
        cross_prob: float = 0.9,
        pmuta_prob: float = 0.01,
        select_prob: float = 0.8,
    ) -> None:
        self.maxgen: int = maxgen  # 最大迭代次数
        self.size_pop: int = size_pop  # 群体个数
        self.cross_prob: float = cross_prob  # 交叉概率
        self.pmuta_prob: float = pmuta_prob  # 变异概率
        self.select_prob: float = select_prob  # 选择概率

        self.data: ArrayF = data  # 城市的坐标数据
        self.num: int = len(data)  # 城市个数 对应染色体长度
        # 城市间距离的矩阵
        self.matrix_distance: ArrayF = self.matrix_dis()
        # 距离矩阵n*n, 第[i,j]个元素表示城市i到j距离matrix_dis函数见下文

        self.select_num: int = max(floor(self.size_pop * self.select_prob + 0.5), 2)
        # 通过选择概率确定子代的选择个数

        self.chrom: ArrayI = np.array([0] * self.size_pop * self.num).reshape(
            self.size_pop, self.num
        )
        self.sub_sel: ArrayI = np.array([0] * self.select_num * self.num).reshape(
            self.select_num, self.num
        )
        # 父代和子代群体的初始化（不直接用np.zeros是为了保证单个染色体的编码为整数，np.zeros对应的数据类型为浮点型）
        # 可以考虑用np.zeros创建矩阵，再用astype(int)转换为整数型
        # self.chrom: ArrayI = np.zeros((self.size_pop, self.num)).astype(
        #     int64
        # )
        # self.sub_sel: ArrayI = np.zeros((self.size_pop, self.num)).astype(
        #     int64
        # )

        # 存储群体中最优染色体的路径总长度，对应单个染色体的适应度就是其倒数
        self.fitness: ArrayF = np.zeros(self.size_pop)

        # 保存每一步的群体的最优路径和距离
        self.best_fit: list[Any] = []
        self.best_path: list[Any] = []

    def matrix_dis(self) -> ArrayF:
        """
        计算城市间的距离函数，得到一个矩阵
        """
        res: ArrayF = np.zeros((self.num, self.num))  # 初始化各点距离矩阵，默认为0.0
        for i in range(self.num):  # 循环选择每个点
            for j in range(i + 1, self.num):  # 再循环选择之后的点
                res[i, j] = np.linalg.norm(
                    self.data[i, :] - self.data[j, :]
                )  # 计算两个点之间的距离，并写入到相应点
                res[j, i] = res[i, j]  # 写入到斜对称的点
        return res

    def rand_chrom(self) -> None:
        """
        随机产生初始化群体函数
        """
        rand_ch: ArrayI = np.array(range(self.num))  # 生成一条初始染色体，对应初始的城市排序
        for i in range(self.size_pop):  # 循环生成size_pop条染色体
            np.random.shuffle(rand_ch)  # 打乱初始排序
            self.chrom[i, :] = rand_ch  # 写入相应位置
            self.fitness[i] = self.comp_fit(rand_ch)  # 将每一条染色体的路径距离值写入fitness矩阵

    def comp_fit(self, one_path: ArrayI) -> float:
        """
        计算单个染色体的路径距离值，可利用该函数更新self.fittness
        """
        # res: ArrayF = np.array(0., dtype=float64)
        res: float = 0.0  # 生成单条染色的路径距离值的总和，默认为0
        for i in range(self.num - 1):  # 循环选择出每一个点，即每一个城市
            # 查询得到相邻两个点之间的距离，并加入总和
            res += float(self.matrix_distance[one_path[i], one_path[i + 1]])
        # 查询得到最后一个点和第一个点之间的距离，并加入总和。可能会删除
        # res += float(self.matrix_distance[one_path[-1], one_path[0]])
        return res

    def out_path(self, one_path: ArrayI) -> str:
        """
        路径可视化函数，注意程序的索引值要比现实减1
        """
        res: str = str(one_path[0] + 1) + "-->"  # 第一个点
        for i in range(1, self.num):  # 加入之后所有点
            res += str(one_path[i] + 1) + "-->"
        # res += str(one_path[0] + 1) + "\n"  # 加入最后一个点和第一个点。可能会删除
        return res
        # print(res)

    def select_sub(self) -> None:
        """
        子代选取，根据选中概率与对应的适应度函数，采用随机遍历选择方法
        """
        fit: ArrayF = 1 / self.fitness  # 适应度等于路径距离值的倒数。获得适应度矩阵
        cumsum_fit: ArrayF = np.cumsum(fit)  # 获得适应度累加的矩阵
        pick: ArrayF = (
            cumsum_fit[-1]
            / self.select_num
            * (np.random.rand() + np.array(range(self.select_num)))
        )
        # 生成从父代染色体群体中选择select_num数量的选中概率矩阵
        i: int = 0  # i是染色体数量的索引，j是选择出的染色体数量的索引
        j: int = 0  # i是染色体数量的索引，j是选择出的染色体数量的索引
        index: list[int] = []  # 选择出的染色体所在索引的列表
        while i < self.size_pop and j < self.select_num:
            # 遍历所有的染色体。如果当前染色体的累计适应度超过当前选择的概率，
            # 则将当前染色体的索引放入选择染色体列表；否则查看下一条
            if cumsum_fit[i] >= pick[j]:
                index.append(i)
                j += 1
            else:
                i += 1
        self.sub_sel = self.chrom[index, :]  # 将选择出的染色体放置到子代群体

    # 交叉，依概率对子代个体进行交叉操作
    def cross_sub(self) -> None:
        # 根据子代选择数量的奇偶，确定子代群体中参与交叉的染色体的索引
        if self.select_num % 2 == 0:
            num = range(0, self.select_num, 2)
        else:
            num = range(0, self.select_num - 1, 2)
        for i in num:
            if self.select_prob > np.random.rand():
                # 如果选择概率超过某个值，则当前的染色体发生交叉
                self.sub_sel[i, :], self.sub_sel[i + 1, :] = self.intercross(
                    self.sub_sel[i, :], self.sub_sel[i + 1, :]
                )

    def intercross(self, ind_a: ArrayI, ind_b: ArrayI) -> tuple[ArrayI, ArrayI]:  # type: ignore
        # 随机生成两个整数作为交叉的基因索引范围，上限是基因数量
        r1: int = np.random.randint(low=0, high=self.num)  # type: ignore
        r2: int = np.random.randint(low=0, high=self.num)  # type: ignore
        while r1 == r2:
            # 避免两个整数是一样的
            r2: int = np.random.randint(low=0, high=self.num)  # type: ignore
        left: int = min(r1, r2)  # 基因索引左边值
        right: int = max(r1, r2)  # 基因索引左边值
        # 复制两条参与交叉的染色体
        ind_a1: ArrayI = ind_a.copy()
        ind_b1: ArrayI = ind_b.copy()
        for i in range(left, right + 1):
            # 再次复制两条参与交叉的染色体
            ind_a2: ArrayI = ind_a.copy()
            ind_b2: ArrayI = ind_b.copy()
            # 交换两条不同染色体上相同索引的基因
            ind_a[i] = ind_b1[i]
            ind_b[i] = ind_a1[i]
            # 查找两条染色体中与当前索引i上的基因相同的其他基因的索引
            # 旨在避免染色体上出现重复的基因
            x: ArrayI = np.argwhere(ind_a == ind_a[i])
            y: ArrayI = np.argwhere(ind_b == ind_b[i])
            # 如果染色体上出现重复基因
            # 则将非当前索引i之外的其他索引的值，都还原为对应染色体上原来的值
            if len(x) == 2:
                ind_a[x[x != i]] = ind_a2[i]
            if len(y) == 2:
                ind_b[y[y != i]] = ind_b2[i]
            return ind_a, ind_b

    def mutation_sub(self) -> None:
        """变异模块"""
        for i in range(self.select_num):
            if np.random.rand() <= self.cross_prob:
                # 随机生成两个整数作为变异的基因索引，上限是基因数量
                r1: int = np.random.randint(low=0, high=self.num)  # type: ignore
                r2: int = np.random.randint(low=0, high=self.num)  # type: ignore
                while r1 == r2:
                    # 避免两个整数是一样的
                    r2: int = np.random.randint(low=0, high=self.num)  # type: ignore
                # 交换当前染色体上两个位置的基因
                self.sub_sel[i, [r1, r2]] = self.sub_sel[i, [r2, r1]]

    def reverse_sub(self) -> None:
        """进化逆转"""
        for i in range(self.select_num):
            # 随机生成两个整数作为交换的基因索引，上限是基因数量
            r1: int = np.random.randint(low=0, high=self.num)  # type: ignore
            r2: int = np.random.randint(low=0, high=self.num)  # type: ignore
            while r1 == r2:
                # 避免两个整数是一样的
                r2: int = np.random.randint(low=0, high=self.num)  # type: ignore
            left: int = min(r1, r2)  # 基因索引左边值
            right: int = max(r1, r2)  # 基因索引右边值
            sel: ArrayI = self.sub_sel[i, :].copy()  # 复制当前染色体
            sel[left : right + 1] = self.sub_sel[i, left : right + 1][
                ::-1
            ]  # 颠倒当前染色体的指定基因范围
            # 如果当前逆转的染色体的适应度超过原来的染色体，则替换为当前染色体
            if self.comp_fit(sel) < self.comp_fit(self.sub_sel[i, :]):
                self.sub_sel[i, :] = sel

    def reins(self) -> None:
        """子代插入父代，得到相同规模的新群体"""
        # 获得倒置的群体中每个染色体的路径总长度的索引矩阵
        index: ArrayI = np.argsort(self.fitness)[::-1]
        # 选择索引矩阵中指定选择数量的索引，并保存为子代
        self.chrom[index[: self.select_num], :] = self.sub_sel

    def draw_path(self, path: ArrayI) -> None:
        ## 绘制路径图
        fig1, ax = plt.subplots()  # type: ignore
        x: ArrayF = self.data[:, 0]
        y: ArrayF = self.data[:, 1]
        ax.scatter(x, y, linewidths=0.1)  # type: ignore
        for i, txt in enumerate(range(1, len(self.data) + 1)):
            ax.annotate(txt, (x[i], y[i]))  # type: ignore
        # res = self.chrom[g]
        # res: ArrayI = self.best_path[g]
        x0: ArrayF = x[path]
        y0: ArrayF = y[path]
        for i in range(len(self.data) - 1):
            plt.quiver(x0[i], y0[i], x0[i + 1] - x0[i], y0[i + 1] - y0[i], color="r", width=0.005, angles="xy", scale=1, scale_units="xy")  # type: ignore
        # plt.quiver(x0[-1], y0[-1],x0[0]-x0[-1], y0[0]-y0[-1], color='r', width=0.005,angles='xy',scale=1, scale_units='xy') # type: ignore
        # plt.show() # type: ignore


In [ ]:
# 路径坐标为
data: ArrayF = np.array(
    [
        16.47,
        96.10,
        16.47,
        94.44,
        20.09,
        92.54,
        22.39,
        93.37,
        25.23,
        97.24,
        22.00,
        96.05,
        20.47,
        97.02,
        17.20,
        96.29,
        16.30,
        97.38,
        14.05,
        98.12,
        16.53,
        97.38,
        21.52,
        95.59,
        19.41,
        97.13,
        20.09,
        92.55,
    ]
).reshape(14, 2)

In [ ]:
Path_short: Gena_TSP = Gena_TSP(data=data, maxgen=200)  # 根据位置坐标，生成一个遗传算法类
Path_short.rand_chrom() # 初始化父类
print(Path_short.out_path(Path_short.chrom[0,:]))
print('初始染色体的路程: '+str( Path_short.fitness[0]))

In [ ]:
#循环迭代遗传过程
for i in range(Path_short.maxgen):
    Path_short.select_sub()   #选择子代
    Path_short.cross_sub()    #交叉
    Path_short.mutation_sub() #变异
    Path_short.reverse_sub()  #进化逆转
    Path_short.reins()        #子代插入
    #重新计算新群体的距离值
    for j in range(Path_short.size_pop):
        Path_short.fitness[j] = Path_short.comp_fit(Path_short.chrom[j,:])  # type: ignore
     
    #每隔四十步显示当前群体的最优路径
    index: int = int(Path_short.fitness.argmin())
    if (i + 1) % 40 == 0:
        print('第'+str(i+1)+'步后的最短的路程: '+str(Path_short.fitness[index]))
        print('第'+str(i+1)+'步后的最优路径:')
        # Path_short.out_path(Path_short.chrom[index,:])# 显示每一步的最优路径
        print(Path_short.out_path(Path_short.chrom[index,:]))# 显示每一步的最优路径
    
    #存储每一步的最优路径及距离
    Path_short.best_fit.append(Path_short.fitness[index]) # type: ignore
    Path_short.best_path.append(Path_short.chrom[index,:]) # type: ignore

print("最优路径：")
print(Path_short.out_path(Path_short.best_path[-1]))
print(f"最优路径的路程：{Path_short.best_fit[-1]}")

In [ ]:
Path_short.draw_path(Path_short.best_path[-1])

## 二、混合粒子群算法

### A. 混合粒子群算法的基本运算过程如下：

1. 初始化编码：设置最大进化代数$T_{max}$、随机生成$m$个染色体的群体编码。
1. 适应度函数：对每一个染色体$x_k$，其个体适应度函数设置为$f(x_k)=D_k$，其中$D_k$表示该条路径的总长度。
1. 交叉：将每个个体与该个体的个体极值和当前群体的群体极值进行交叉操作更新，只有交叉后的新个体比旧个体的适应度更好，才替换更改。
1. 变异：对单个染色体随机交换两个点的位置，如果变异后的个体比旧个体的适应度更好，就替换更改。

### B. python程序实现

构建一个类保存混合粒子群算法的初始化参数和函数计算

In [ ]:
from typing import TypeAlias
import numpy as np
from numpy import float64, int64
import numpy.typing as npt
import matplotlib.pyplot as plt  # 导入所需要的库

In [ ]:
ArrayF: TypeAlias = npt.NDArray[float64]
ArrayI: TypeAlias = npt.NDArray[int64]

plt.style.use("seaborn-v0_8")

In [ ]:
class Hybrid_POS_TSP:
    def __init__(self, data: ArrayF, num_pop: int = 200) -> None:
        self.num_pop: int = num_pop  # 群体个数
        self.data: ArrayF = data        # 城市坐标
        self.num: int =len(data)     # 城市个数

        # 群体的初始化和路径的初始化         
        self.chrom: ArrayI = np.array([0] * self.num_pop * self.num).reshape(self.num_pop, self.num)
        self.fitness: list[float] = [0.0] * self.num_pop

        # 保存每一步的群体的最优路径和距离
        self.best_fit: list[float] = []
        self.best_chrom: list[ArrayI] = []
        
        # 路径矩阵，函数matrix_dis同遗传算法
        self.matrix_distance: ArrayF = self.matrix_dis()

    def matrix_dis(self) -> ArrayF:
        """
        计算城市间的距离函数，得到一个矩阵
        """
        res: ArrayF = np.zeros((self.num, self.num))  # 初始化各点距离矩阵，默认为0.0
        for i in range(self.num):  # 循环选择每个点
            for j in range(i + 1, self.num):  # 再循环选择之后的点
                res[i, j] = np.linalg.norm(
                    self.data[i, :] - self.data[j, :]
                )  # 计算两个点之间的距离，并写入到相应点
                res[j, i] = res[i, j]  # 写入到斜对称的点
        return res

    def rand_chrom(self) -> None:
        """
        随机产生初始化群体函数
        """
        rand_ch: ArrayI = np.array(range(self.num))  # 生成一条初始染色体，对应初始的城市排序
        for i in range(self.num_pop):  # 循环生成size_pop条染色体
            np.random.shuffle(rand_ch)  # 打乱初始排序
            self.chrom[i, :] = rand_ch  # 写入相应位置
            self.fitness[i] = self.comp_fit(rand_ch)  # 将每一条染色体的路径距离值写入fitness矩阵

    def comp_fit(self, one_path: ArrayI) -> float:
        """
        计算单个染色体的路径距离值，可利用该函数更新self.fittness
        """
        # res: ArrayF = np.array(0., dtype=float64)
        res: float = 0.0  # 生成单条染色的路径距离值的总和，默认为0
        for i in range(self.num - 1):  # 循环选择出每一个点，即每一个城市
            # 查询得到相邻两个点之间的距离，并加入总和
            res += float(self.matrix_distance[one_path[i], one_path[i + 1]])
        # 查询得到最后一个点和第一个点之间的距离，并加入总和。可能会删除
        # res += float(self.matrix_distance[one_path[-1], one_path[0]])
        return res

    def out_path(self, one_path: ArrayI) -> str:
        """
        路径可视化函数，注意程序的索引值要比现实减1
        """
        res: str = str(one_path[0] + 1) + "-->"  # 第一个点
        for i in range(1, self.num):  # 加入之后所有点
            res += str(one_path[i] + 1) + "-->"
        # res += str(one_path[0] + 1) + "\n"  # 加入最后一个点和第一个点。可能会删除
        return res
        # print(res)

    def cross_1(self, path: ArrayI, best_path: ArrayI) -> ArrayI:
        '''两条路径的交叉函数'''
        r1: int = np.random.randint(self.num) # type: ignore
        r2: int = np.random.randint(self.num) # type: ignore
        while r2 == r1:
            r2 = np.random.randint(self.num) # type: ignore

        left: int = min(r1, r2)
        right: int = max(r1, r2)
        cross: ArrayI = best_path[left:right + 1]
        for i in range(right - left + 1):
            for k in range(self.num):
                if path[k] == cross[i]:
                    path[k:self.num - 1] = path[k + 1:self.num]
                    path[-1] = 0
        path[self.num - right + left - 1:self.num] = cross
        return path

    def mutation(self, path: ArrayI) -> ArrayI:
        '''变异'''
        r1: int = np.random.randint(self.num) # type: ignore
        r2: int = np.random.randint(self.num) # type: ignore
        while r2 == r1:
            r2: int = np.random.randint(self.num) # type: ignore
        path[r1], path[r2] = path[r2], path[r1]
        return path

    def draw_path(self, path: ArrayI) -> None:
        ## 绘制路径图
        fig1, ax = plt.subplots()  # type: ignore
        x: ArrayF = self.data[:, 0]
        y: ArrayF = self.data[:, 1]
        ax.scatter(x, y, linewidths=0.1)  # type: ignore
        for i, txt in enumerate(range(1, len(self.data) + 1)):
            ax.annotate(txt, (x[i], y[i]))  # type: ignore
        res = path
        x0: ArrayF = x[res]
        y0: ArrayF = y[res]
        for i in range(len(self.data) - 1):
            plt.quiver(x0[i], y0[i], x0[i + 1] - x0[i], y0[i + 1] - y0[i], color="r", width=0.005, angles="xy", scale=1, scale_units="xy")  # type: ignore
        # plt.quiver(x0[-1], y0[-1],x0[0]-x0[-1], y0[0]-y0[-1], color='r', width=0.005,angles='xy',scale=1, scale_units='xy') # type: ignore
        # plt.show() # type: ignore


In [ ]:
# 路径坐标为
data: ArrayF = np.array(
    [
        16.47,
        96.10,
        16.47,
        94.44,
        20.09,
        92.54,
        22.39,
        93.37,
        25.23,
        97.24,
        22.00,
        96.05,
        20.47,
        97.02,
        17.20,
        96.29,
        16.30,
        97.38,
        14.05,
        98.12,
        16.53,
        97.38,
        21.52,
        95.59,
        19.41,
        97.13,
        20.09,
        92.55,
    ]
).reshape(14, 2)

In [ ]:
Path_short: Hybrid_POS_TSP = Hybrid_POS_TSP(data) #混合粒子群算法类
Path_short.rand_chrom() #初始化种群

print(Path_short.chrom)
print('初始染色体的路程: '+str( Path_short.fitness[0]))


In [ ]:
Path_short.draw_path(Path_short.chrom[0])

In [ ]:
#存储个体极值的路径和距离
best_P_chrom: ArrayI = Path_short.chrom.copy()
best_P_fit: list[float] = Path_short.fitness.copy()

min_index: int = int(np.argmin(Path_short.fitness)) # 获得距离最短的路径所在的索引

# 存储当前一代的种群极值的路径和距离
best_G_chrom: ArrayI = Path_short.chrom[min_index, :]
best_G_fit: float = Path_short.fitness[min_index]

# 存储每一步迭代后的最优路径和距离
Path_short.best_chrom.append(best_G_chrom)
Path_short.best_fit.append(best_G_fit)

# 复制当前群体进行交叉变异
x_new: ArrayI = Path_short.chrom.copy()


In [ ]:
# 迭代
for i in range(200):
    # 更新当前的个体极值
    for j in range(Path_short.num_pop):
        # 如果每条染色体的距离小于
        if Path_short.fitness[j] < best_P_fit[j]:
            best_P_fit[j] = Path_short.fitness[j]
            best_P_chrom[j,:] = Path_short.chrom[j,:]
    # 更新当前种群的群体极值
    min_index = int(np.argmin(Path_short.fitness))
    best_G_chrom: ArrayI = Path_short.chrom[min_index,:]
    best_G_fit: float = Path_short.fitness[min_index]
    
    # 更新每一步迭代后的全局最优路径和解
    if best_G_fit < Path_short.best_fit[-1]:
        Path_short.best_fit.append(best_G_fit)
        Path_short.best_chrom.append(best_G_chrom)
    else:
        Path_short.best_fit.append(Path_short.best_fit[-1])
        Path_short.best_chrom.append(Path_short.best_chrom[-1])
     
    # 将每个个体与个体极值和当前的群体极值进行交叉
    for j in range(Path_short.num_pop):
        # 与个体极值交叉
        x_new[j,:] = Path_short.cross_1(x_new[j,:], best_P_chrom[j,:]) # type: ignore
        fit = Path_short.comp_fit(x_new[j,:])
        # 判断是否保留
        if fit < Path_short.fitness[j]:
            Path_short.chrom[j,:] = x_new[j,:]
            Path_short.fitness[j] = fit
        # 与当前极值交叉            
        x_new[j,:] = Path_short.cross_1(x_new[j,:], best_G_chrom) # type: ignore
        fit: float = Path_short.comp_fit(x_new[j,:])
        if fit < Path_short.fitness[j]:
            Path_short.chrom[j,:] = x_new[j,:]
            Path_short.fitness[j] = fit
        # 变异
        x_new[j,:] = Path_short.mutation(x_new[j,:]) # type: ignore
        fit: float = Path_short.comp_fit(x_new[j,:])   
        if fit <= Path_short.fitness[j]:
            Path_short.chrom[j] = x_new[j,:]
            Path_short.fitness[j] = fit
            
    if (i + 1) % 20 == 0:
        print('第'+str(i + 1)+'步后的最短的路程: '+str(Path_short.fitness[min_index]))
        print('第'+str(i + 1)+'步后的最优路径:')
        print(Path_short.out_path(Path_short.chrom[min_index,:])) # 显示每一步的最优路径


In [ ]:
Path_short.draw_path(Path_short.best_chrom[-1])

## 三、模拟退火算法

### A. 模拟退火算法的基本运算过程如下：

1. 控制参数：设置初始温度$T$，终止温度$T_{end}$，链长$L$以及降温速率$r$。
1. 初始解：随机产生一个初始路径。
1. 变换：对当前个体$S_1$随机交换两个点的位置，得到新的个体$S_2$。
1. Metropolis准则：设置路径函数为$fit(S)$，则$S1, S2$的路径差$df=fit(S2)-fit(S1)$，Metropolis准则为：

$$
\begin{equation*}
P = \begin{cases}
1, &df < 0\\
\text{exp}(\frac{-df}{T}), &df > 0
\end{cases}
\end{equation*}
$$

以概率$P$来选择是否接受新的路径：如过$df < 0$，则接受新的路径$S_2$，否则以概率$\text{exp}(-df/T)$接受新的路径。

5. 降温 ：利用降温速率$r$进行降温，$T=rT$，直至$T < T_{end}$停止迭代。

### B. python程序实现

构建一个类保存模拟退火算法的初始化参数和函数计算：

In [ ]:
from typing import TypeAlias, Any
import numpy as np
from numpy import float64, int64
import numpy.typing as npt
import matplotlib.pyplot as plt  # 导入所需要的库

In [ ]:
ArrayF: TypeAlias = npt.NDArray[float64]
ArrayI: TypeAlias = npt.NDArray[int64]

plt.style.use("seaborn-v0_8")

In [ ]:
class SA_TSP:
    def __init__(self, data: ArrayF, T: int | float =1000, T_end: float = 1e-3, L: int = 200, r: float = 0.9) -> None:
        self.T: int | float = T  #初始温度
        self.T_end: float = T_end  #终止温度
        self.data: ArrayF = data #位置坐标
        self.num: int = len(data) #城市个数
        self.L: int = L      # 每一个温度下的链长
        self.r: float = r  #降温速率
        #距离矩阵
        self.matrix_distance: ArrayF = self.matrix_dis()

        
        self.chrom: ArrayI = np.array([0] * self.num) #初始化路径和距离
        self.fitness = 0.0
        # self.fitness: ArrayF = np.array([0.0] * self.num)

        self.new_chrom: ArrayI = np.array([0] * self.num) #变换后的路径和距离
        # self.new_fitness: ArrayF = np.array([0.0] * self.num)
        self.new_fitness = 0.0

        self.best_chrom: list[ArrayI] = []
        self.best_fit: list[Any] = []

    def matrix_dis(self) -> ArrayF:
        """
        计算城市间的距离函数，得到一个矩阵
        """
        res: ArrayF = np.zeros((self.num, self.num))  # 初始化各点距离矩阵，默认为0.0
        for i in range(self.num):  # 循环选择每个点
            for j in range(i + 1, self.num):  # 再循环选择之后的点
                res[i, j] = np.linalg.norm(
                    self.data[i, :] - self.data[j, :]
                )  # 计算两个点之间的距离，并写入到相应点
                res[j, i] = res[i, j]  # 写入到斜对称的点
        return res

    # def rand_chrom(self) -> None:
    #     """
    #     随机产生初始化群体函数
    #     """
    #     rand_ch: ArrayI = np.array(range(self.num))  # 生成一条初始染色体，对应初始的城市排序
    #     for i in range(self.L):  # 循环生成L条染色体
    #         np.random.shuffle(rand_ch)  # 打乱初始排序
    #         # self.chrom[i, :] = rand_ch  # 写入相应位置
    #         self.chrom[i] = rand_ch  # 写入相应位置
    #         self.fitness[i] = self.comp_fit(rand_ch)  # 将每一条染色体的路径距离值写入fitness矩阵

    def rand_chrom(self) -> None:
        """
        随机产生初始化群体函数
        """
        rand_ch: ArrayI = np.array(range(self.num))  # 生成一条初始染色体，对应初始的城市排序
        np.random.shuffle(rand_ch)  # 打乱初始排序
        self.chrom = rand_ch  # 写入相应位置
        self.fitness = self.comp_fit(rand_ch)  # 将每一条染色体的路径距离值写入fitness

    def comp_fit(self, one_path: ArrayI) -> float:
        """
        计算单个染色体的路径距离值，可利用该函数更新self.fittness
        """
        # res: ArrayF = np.array(0., dtype=float64)
        res: float = 0.0  # 生成单条染色的路径距离值的总和，默认为0
        for i in range(self.num - 1):  # 循环选择出每一个点，即每一个城市
            # 查询得到相邻两个点之间的距离，并加入总和
            res += float(self.matrix_distance[one_path[i], one_path[i + 1]])
        # 查询得到最后一个点和第一个点之间的距离，并加入总和。可能会删除
        # res += float(self.matrix_distance[one_path[-1], one_path[0]])
        return res

    def out_path(self, one_path: ArrayI) -> str:
        """
        路径可视化函数，注意程序的索引值要比现实减1
        """
        res: str = str(one_path[0] + 1) + "-->"  # 第一个点
        for i in range(1, self.num):  # 加入之后所有点
            res += str(one_path[i] + 1) + "-->"
        # res += str(one_path[0] + 1) + "\n"  # 加入最后一个点和第一个点。可能会删除
        return res
        # print(res)

    def new_way_1(self) -> None:
        '''更新交换后的路径和距离'''
        self.new_chrom = self.chrom.copy()
        r1: int = np.random.randint(self.num) # type: ignore
        r2: int = np.random.randint(self.num) # type: ignore
        while r2 == r1:
            r2: int = np.random.randint(self.num) # type: ignore
        self.new_chrom[r1], self.new_chrom[r2] =  self.new_chrom[r2], self.new_chrom[r1]
        self.new_fitness: float = self.comp_fit(self.new_chrom)

    def metropolis(self) -> None:
        '''更新交换后的路径和距离'''
        ds: float = self.new_fitness-self.fitness
        if ds < 0:
            #新路径更短 接受为新解
            self.chrom = self.new_chrom.copy() #数组直接赋值会共享内存 采用copy避免
            self.fitness: float = self.new_fitness
        else:
            if np.exp(-ds / self.T) > np.random.rand():
                self.chrom = self.new_chrom.copy()
                self.fitness: float = self.new_fitness

    def draw_path(self, path: ArrayI) -> None:
        ## 绘制路径图
        fig1, ax = plt.subplots()  # type: ignore
        x: ArrayF = self.data[:, 0]
        y: ArrayF = self.data[:, 1]
        ax.scatter(x, y, linewidths=0.1)  # type: ignore
        for i, txt in enumerate(range(1, len(self.data) + 1)):
            ax.annotate(txt, (x[i], y[i]))  # type: ignore
        res = path
        x0: ArrayF = x[res]
        y0: ArrayF = y[res]
        for i in range(len(self.data) - 1):
            plt.quiver(x0[i], y0[i], x0[i + 1] - x0[i], y0[i + 1] - y0[i], color="r", width=0.005, angles="xy", scale=1, scale_units="xy")  # type: ignore
        # plt.quiver(x0[-1], y0[-1],x0[0]-x0[-1], y0[0]-y0[-1], color='r', width=0.005,angles='xy',scale=1, scale_units='xy') # type: ignore
        # plt.show() # type: ignore


In [ ]:
# 路径坐标为
data: ArrayF = np.array(
    [
        16.47,
        96.10,
        16.47,
        94.44,
        20.09,
        92.54,
        22.39,
        93.37,
        25.23,
        97.24,
        22.00,
        96.05,
        20.47,
        97.02,
        17.20,
        96.29,
        16.30,
        97.38,
        14.05,
        98.12,
        16.53,
        97.38,
        21.52,
        95.59,
        19.41,
        97.13,
        20.09,
        92.55,
    ]
).reshape(14, 2)

In [ ]:
Path_short = SA_TSP(data, T=5000, r=0.9, L=400) 
Path_short.rand_chrom()  #初始化路径

In [ ]:
print(Path_short.out_path(Path_short.chrom))

In [ ]:
print("最佳染色体的路径：")
print(Path_short.out_path(Path_short.chrom))
print('初始染色体的路程: '+str( Path_short.fitness))
# 存储最退火过程中的最优路径变化
Path_short.best_chrom = [Path_short.chrom]
Path_short.best_fit = [Path_short.fitness]

while Path_short.T > Path_short.T_end:
    chrom: list[Any] = []
    fit: list[Any] = []   #存储每一个退火过程的路径和距离找寻最优

    for i in range(Path_short.L):
        Path_short.new_way_1()  #变换产生新路径
        Path_short.metropolis()  #判断是否接受新路径
        
        chrom.append(Path_short.chrom)
        fit.append(Path_short.fitness)
    
    #存储该步迭代后的最优路径
    index: int = int(np.argmin(fit))
    if fit[index] >= Path_short.best_fit[-1]:
        Path_short.best_fit.append(Path_short.best_fit[-1])
        Path_short.best_chrom.append(Path_short.best_chrom[-1])
    else:
        Path_short.best_chrom.append(chrom[index])
        Path_short.best_fit.append(fit[index])
        
    #更新温度
    Path_short.T *= Path_short.r

In [ ]:
print("最佳染色体的路径：")
print(Path_short.out_path(Path_short.best_chrom[-1]))
print('最佳染色体的路程: '+str( Path_short.best_fit[-1]))

In [ ]:
Path_short.draw_path(Path_short.best_chrom[-1])

## 四、蚁群算法

### A. 蚁群算法算法的求解过程如下：

1. 参数初始化：蚁群规模，信息素重要程度因子$\alpha$，启发函数重要程度因子$\beta$，信息素挥发因子$\rho$，信息素释放总量$Q$，最大迭代次数$max\_iter$。
1. 构建解空间：将每个蚂蚁随机置于不同的出发点，对第$k$个蚂蚁按照概率

$$
\begin{equation*}
P_{ij}^{k} = \begin{cases}
\frac{\tau_{ij}^{\alpha}\cdot \eta_{ij}^{\beta}}{\sum_{r \in allow_k}\tau_{ir}^{\alpha}\cdot\eta_{ir}^{\beta}}, &j \in allow_{k}\\
0, &other
\end{cases}
\end{equation*}
$$

选择下一个访问城市，其中$i$表示蚂蚁$k$当前处于的城市，$j$表示下一个访问城市，$\tau_{ij}$表示当前城市$i\rightarrow j$连接路径的信息素浓度，$\eta_{ij}=\frac{1}{d_{ij}}$表示城市$i\rightarrow j$的期望程度，$d_{ij}$表示$i\rightarrow j$的距离，$allow_k$表示蚂蚁未访问的城市集合。
3. 更新信息素：计算各个蚂蚁结果的路程，记录最优路径，根据式子更新信息素浓度矩阵$\tau_{ij}$，更新方式采用ant cycle system模型$\tau_{ij}=(1-\rho)\tau_{ij}+\sum_{k=1}^{n}\Delta_{ij}^{k},\, \rho \in (0, 1)$。其中

$$
\begin{equation*}
\Delta_{ij}^{k} = \begin{cases}
Q/fit(k), &第k只蚂蚁从城市访问到j\\
0, &other
\end{cases}
\end{equation*}
$$

$fit(k)$表示蚂蚁$k$结果的路径和。

4. 若迭代次数小于$max\_iter$，清空蚂蚁路径记录，重复2-4。

### B. Python程序实现

构建一个类蚁群算法的初始化参数和函数计算：